In [ ]:
import akshare as ak
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import json
import requests
import os
import re
import datetime
import pytz
import plotly.graph_objects as go
import sys

from tqdm import tqdm
from scipy import stats

# get_keys_openai()
pd.set_option("display.max_columns", 800, "display.max_rows", 20)
cwd = os.getcwd()
import polars as pl
from samoyan_pack.connector_ import connect_db
import time
import chinese_calendar as cc

engine = connect_db("samo_test", "mysql").connect_()

In [ ]:
def load_etl_data(_engine):
    ## * load neccessary data.
    # ods_futures_cbot_di = pl.read_database(
    #     "select * from ods_futures_cbot_di where mark = 'soybean'", _engine
    # )
    ods_futures_dce_hist_price_daily_di = pl.read_database(
        "select * from ods_futures_dce_hist_price_daily_di where variety = 'M'", _engine
    )
    # ods_futures_dce_spot_100ppi_di = pl.read_database(
    #     "select * from ods_futures_dce_spot_100ppi_di where symbol is not null", _engine
    # )
    # ods_min_future_price_mi = pl.read_database('select * from ods_min_future_price_mi', _engine)
    # ods_spot_price_corn_feedtrade_di = pl.read_database(
    #     "select * from ods_spot_price_corn_feedtrade_di", _engine
    # )
    # ods_spot_price_sbmeal_feedtrade_di = pl.read_database(
    #     "select * from ods_spot_price_sbmeal_feedtrade_di", _engine
    # )
    # # # ods_min_future_price_mi['datetime']  = pd.to_datetime(ods_min_future_price_mi['datetime'])
    return (
        # ods_futures_cbot_di,
        ods_futures_dce_hist_price_daily_di,
        # ods_futures_dce_spot_100ppi_di,
        # ods_spot_price_corn_feedtrade_di,
        # ods_spot_price_sbmeal_feedtrade_di,
    )


(
    # ods_futures_cbot_di,
    ods_futures_dce_hist_price_daily_di,
    # ods_futures_dce_spot_100ppi_di,
    # ods_min_future_price_mi,
    # ods_spot_price_corn_feedtrade_di,
    # ods_spot_price_sbmeal_feedtrade_di,
) = load_etl_data(engine)

In [ ]:
ods_futures_dce_hist_price_daily_di = (
    ods_futures_dce_hist_price_daily_di.with_columns(
        pl.col("timestamp").str.to_datetime().cast(pl.Date)
    )
    .drop(["variety", "contract_month", "商品商品", "pk"])
    .rename({"turnover": "amount"})
)

In [9]:
from model import Kronos, KronosTokenizer, KronosPredictor

# Load from Hugging Face Hub
tokenizer = KronosTokenizer.from_pretrained("NeoQuasar/Kronos-Tokenizer-base")
model = Kronos.from_pretrained("D:\\samo\\Kronos\\modeldown\\Kronos-mini")

config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

Loading weights from local directory


In [10]:
# Initialize the predictor
predictor = KronosPredictor(model, tokenizer, device="cpu", max_context=512)

In [19]:
df = ods_futures_dce_hist_price_daily_di.filter(pl.col("symbol") == "m2601")
df = df.to_pandas()
print(df.shape)

(194, 11)


In [21]:
lookback = 120
pred_len = 194 - 120

# Prepare inputs for the predictor
x_df = df.loc[: lookback - 1, ["open", "high", "low", "close", "volume", "amount"]]
x_timestamp = df.loc[: lookback - 1, "timestamp"]
y_timestamp = df.loc[lookback : lookback + pred_len - 1, "timestamp"]

In [ ]:
# Generate predictions
pred_df = predictor.predict(
    df=x_df,
    x_timestamp=x_timestamp,
    y_timestamp=y_timestamp,
    pred_len=pred_len,
    T=1.0,          # Temperature for sampling
    top_p=0.9,      # Nucleus sampling probability
    sample_count=1  # Number of forecast paths to generate and average
)

print("Forecasted Data Head:")
print(pred_df.head())
pred_df =  pred_df.reset_index()

100%|██████████| 74/74 [00:02<00:00, 25.08it/s]

Forecasted Data Head:
                   open         high          low        close       volume  \
timestamp                                                                     
2025-07-18  3031.123047  3050.507812  3028.334717  3041.871826  267292.5000   
2025-07-21  3020.889404  3037.142334  2977.460938  2992.924316  308835.7500   
2025-07-22  3029.114746  3051.260742  2940.173340  2965.286865  382054.9375   
2025-07-23  2928.066650  3022.598633  2931.990967  3001.527832  305834.2500   
2025-07-24  3002.073975  3074.453369  2951.054443  2973.005615  503579.1875   

                  amount  
timestamp                 
2025-07-18  8.223359e+05  
2025-07-21  9.328933e+05  
2025-07-22  1.118459e+06  
2025-07-23  8.460420e+05  
2025-07-24  1.504668e+06  


In [31]:
f = go.Figure()
f.add_trace(go.Line(x=df["timestamp"], y=df["high"], name="actual"))
f.add_trace(go.Line(x=pred_df["timestamp"], y=pred_df["high"], name="pred"))

# Correctly set the hovermode to 'x unified'
f.update_layout(hovermode='x unified')
